In [27]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
from ydata_profiling import ProfileReport
import requests
import re
# Set up Selenium WebDriver with headless mode

options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--disable-gpu")  # Disable GPU hardware acceleration
options.add_argument("--window-size=1920x1080")  # Set a large window size to avoid hidden elements
driver = webdriver.Chrome(options=options)


In [28]:
# url = 'https://search.nih.gov/search/docs?affiliate=hip&dc=2370&query=malaria'
data = []

BASE_URL = 'https://www.nih.gov'

def scrape_nih():
    url = "https://www.nih.gov/news-events/news-releases"
    driver.get(url)
    time.sleep(2)
    # Get the page source
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')   
    # Get the title
    titles = soup.find_all('li', class_ = "views-row")

    for title in titles:
        # title_text = title.find('h4', class_='teaser-title').text
        article_link = title.find('h4', class_="teaser-title").find('a')['href']
        article_link = BASE_URL + article_link
        pub_date = title.find('span', class_='date-display-single').text
        article_summary = title.find('p', class_='teaser-description').text
        article_summary_without_date = article_summary.replace(pub_date, '')
        article_summary = article_summary_without_date
        # featured_media = title.find('div', class_='featured-media').find('picture')['src']
        

        # get article content
        driver.get(article_link)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        article_title = soup.find('h1').text

        featured_media = soup.find_all('picture')
        if len(featured_media) == 0:
            featured_media = 'Nan'
        else:
            featured_media = featured_media[0].find('img')['src']

        publication_date = soup.find('span', class_='date-display-single').text
        article_content = soup.find_all('p')
        article_content = [p.text for p in article_content]
        article_content = ' '.join(article_content)
        title_text = soup.find('p', class_= 'subtitle').text
        article_content_without_date = article_content.replace(publication_date, '')
        article_content_without_quick_links = article_content_without_date.replace('More » Search Health Topics Quick Links More » Quick Links More » Quick Links More » Quick Links More » Quick Links More » Quick Links News Release', '')
        article_content = article_content_without_quick_links


        article_content_without_media = article_content.replace(featured_media, '')
        article_content = article_content_without_media
        


        data.append({
            'title': article_title,
            'article_link': article_link,
            'pub_date': publication_date,
            'article_summary': article_summary,
            'featured_media': featured_media,
            'article_content': article_content
        })
    df = pd.DataFrame(data)
    df.to_csv('nih_articles.csv')
    return df

# years = ['2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010']

# for yr in years:
#     link = f'https://www.nih.gov/news-events/news-releases?{yr}'
#     scrape_nih(link + '&year=' + yr)


In [ ]:
from selenium import webdriver  
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import WebDriverWait  
from selenium.webdriver.support import expected_conditions as EC  
from bs4 import BeautifulSoup  
import time  

# Initialize the WebDriver  
driver = webdriver.Chrome()  # Ensure ChromeDriver is in your PATH  

url = "https://www.nih.gov/news-events/news-releases"  
BASE_URL = 'https://www.nih.gov'

def scrape_nih():  
    """Scrapes detailed news articles from the NIH news page."""  
    links = []
    driver.get(url)  
    time.sleep(2)  
    html = driver.page_source  
    soup = BeautifulSoup(html, 'html.parser')  
    titles = soup.find_all('li', class_="views-row")  
    nih_next_page(titles).extend(titles)

    data = []  
    article_titles = []


    for title in titles:  
        try:
            article_link = title.find('h4', class_="teaser-title").find('a')['href']  
            article_link = BASE_URL + article_link  
            pub_date = title.find('span', class_='date-display-single').text  
            article_summary = title.find('p', class_='teaser-description').text.replace(pub_date, '')  
            
            # Navigate to article page  
            driver.get(article_link)  
            time.sleep(2)  
            html = driver.page_source  
            soup = BeautifulSoup(html, 'html.parser')  
            article_title = soup.find('h1').text  
            article_titles.append(article_title)

            featured_media = soup.find_all('picture')  
            featured_media = featured_media[0].find('img')['src'] if featured_media else 'Nan'  

            article_content = ' '.join([p.text for p in soup.find_all('p')])  
            article_content = article_content.replace(pub_date, '')  
            article_content = article_content.replace(featured_media, '')  
            
            data.append({  
                'title': article_title,  
                'article_link': article_link,  
                'pub_date': pub_date,  
                'article_summary': article_summary,  
                'featured_media': featured_media,  
                'article_content': article_content  
            })  
        except Exception as e:
            print(f"Error scraping article: {e}")

    df = pd.DataFrame(data)  
    df.to_csv('nih_articles.csv', index=False)  
    print(type(article_titles))
    print(len(article_titles))
    return article_titles 

def nih_next_page(titles):  
    """Scrapes news release titles from all pages until the 'next' button is disabled."""  
    driver.get(url)  
    time.sleep(2)  # Wait for the initial page to load  

    all_titles = []  

    while True:  
        # Scrape titles from the current page  
        # data = scrape_nih()
        # titles = data['title']
        # all_titles.extend(titles)

        # titles = scrape_nih()  
        all_titles.extend(titles)  
        print(f"Scraped {len(titles)} titles from {driver.current_url}")  

        try:  
            # Find the "next" button  
            next_button = WebDriverWait(driver, 10).until(  
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'li.pager__item--next a'))  
            )  

            # Scroll the page to ensure the "next" button is visible  
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)  
            time.sleep(1)  # Wait for the page to settle  

            # Click the "next" button using JavaScript  
            driver.execute_script("arguments[0].click();", next_button)  

            # Wait for the new content to load  
            WebDriverWait(driver, 10).until(  
                EC.presence_of_element_located((By.CSS_SELECTOR, 'h4.teaser-title'))  
            )  
            print("New page content loaded.")  
            time.sleep(2)  # Additional delay to ensure the page is fully loaded  

        except Exception as e:  
            print(f"No more 'next' button found or other error: {e}")  
            break  # Exit the loop if the "next" button is not found or clickable  

    print("Scraping completed for all pages.")  
    return all_titles  

if __name__ == '__main__':  
    all_news = scrape_nih()  
    print(f"Total news releases scraped: {len(all_news)}")  
    # Optionally print the titles or save them to a file.  
    # for title in all_news:  
    #     print(title)  

    driver.quit()  # Close the browser when finished.

Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-

In [29]:
df = scrape_nih()
df.head()

,title,article_link,pub_date,article_summary,featured_media,article_content
0,NIH-funded clinical trial will evaluate new de...,https://www.nih.gov/news-events/news-releases/...,"Tuesday, February 11, 2025",— \n Dengue virus sickens as man...,https://www.nih.gov/sites/default/files/styles...,\n A clinical trial supported by the Nationa...
1,Single dose of broadly neutralizing antibody p...,https://www.nih.gov/news-events/news-releases/...,"Tuesday, February 11, 2025",— \n NIH science lays groundwork...,https://www.nih.gov/sites/default/files/styles...,More » Search Health Topics Quick Links More »...
2,NIH study finds infection-related hospitalizat...,https://www.nih.gov/news-events/news-releases/...,"Tuesday, February 11, 2025",— \n Findings highlight the impo...,Nan,\nFindings highlight the importance of infec...
3,Influenza A viruses adapt shape in response to...,https://www.nih.gov/news-events/news-releases/...,"Monday, February 10, 2025",— \n NIH study identifies previo...,https://www.nih.gov/sites/default/files/styles...,More » Search Health Topics Quick Links More »...
4,Therapy helps peanut-allergic kids tolerate ta...,https://www.nih.gov/news-events/news-releases/...,"Monday, February 10, 2025",— \n NIH trial informs potential...,https://www.nih.gov/sites/default/files/styles...,\nNIH trial informs potential treatment stra...


In [31]:
len(df['title'])

10

In [50]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
from ydata_profiling import ProfileReport
import requests
import re
# Set up Selenium WebDriver with headless mode

options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--disable-gpu")  # Disable GPU hardware acceleration
options.add_argument("--window-size=1920x1080")  # Set a large window size to avoid hidden elements
driver = webdriver.Chrome(options=options)

url = "https://www.nih.gov/news-events/news-releases"  
BASE_URL = 'https://www.nih.gov'

def scrape_nih():  
    """Scrapes detailed news articles from the NIH news page."""  
    driver.get(url)  
    time.sleep(2)  
    html = driver.page_source  
    soup = BeautifulSoup(html, 'html.parser')  
    titles = soup.find_all('li', class_="views-row")  

    # Get titles from pagination
    titles.extend(nih_next_page())

    data = []  
    article_titles = []

    for title in titles:  
        try:
            article_link = title.find('h4', class_="teaser-title").find('a')['href']  
            article_link = BASE_URL + article_link  
            pub_date = title.find('span', class_='date-display-single').text  
            article_summary = title.find('p', class_='teaser-description').text.replace(pub_date, '')  
            
            # Navigate to article page  
            driver.get(article_link)  
            time.sleep(2)  
            html = driver.page_source  
            soup = BeautifulSoup(html, 'html.parser')  
            article_title = soup.find('h1').text  
            article_titles.append(article_title)

            featured_media = soup.find_all('picture')  
            featured_media = featured_media[0].find('img')['src'] if featured_media else 'Nan'  

            article_content = ' '.join([p.text for p in soup.find_all('p')])  
            article_content = article_content.replace(pub_date, '')  
            article_content = article_content.replace(featured_media, '')  
            
            data.append({  
                'title': article_title,  
                'article_link': article_link,  
                'pub_date': pub_date,  
                'article_summary': article_summary,  
                'featured_media': featured_media,  
                'article_content': article_content  
            })  
        except Exception as e:
            print(f"Error scraping article: {e}")

    df = pd.DataFrame(data)  
    df.to_csv('nih_articles.csv', index=False)  
    print(f"Total articles scraped: {len(article_titles)}")
    return article_titles  

def nih_next_page():  
    """Scrapes news articles from all available pages."""  
    all_titles = []  
    prev_length = 0

    while True:  
        html = driver.page_source  
        soup = BeautifulSoup(html, 'html.parser')  
        new_titles = soup.find_all('li', class_="views-row")
        
        all_titles.extend(new_titles)  

        print(f"Scraped {len(new_titles)} titles from {driver.current_url}")  

        try:  
            next_button = WebDriverWait(driver, 10).until(  
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'li.pager__item--next a'))  
            )  

            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)  
            time.sleep(1)  

            driver.execute_script("arguments[0].click();", next_button)  

            WebDriverWait(driver, 10).until(  
                EC.presence_of_element_located((By.CSS_SELECTOR, 'h4.teaser-title'))  
            )  
            print("New page content loaded.")  
            time.sleep(2)  

            if len(all_titles) == prev_length:  # Break if no new articles are found
                break
            prev_length = len(all_titles)

        except Exception as e:  
            print(f"No more 'next' button found or error: {e}")  
            break  

    print(f"Total titles scraped from all pages: {len(all_titles)}")  
    return all_titles  

if __name__ == '__main__':  
    all_news = scrape_nih()  
    print(f"Total news releases scraped: {len(all_news)}")  
    driver.quit()


Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-releases
New page content loaded.
Scraped 10 titles from https://www.nih.gov/news-events/news-